###POLISCI 88 FA 21

###Lab #: Peace Time: Cease-Fire Agreements and the Durability of Peace

####Due Date: X

###Binary Logistic Regression

Logistic Regression is a (Machine Learning) classification algorithm which is used when we want to predict a categorical variable (Yes/No, Pass/Fail) based on a set of independent variable(s).

In the Logistic Regression model, the log of odds of the dependent variable is modeled as a linear combination of the independent variables.

We can get more clarity on Binary Logistic Regression using a practical example. For this lab we will be using the `[statsmodels]`(https://www.statsmodels.org/stable/index.html) python package similar to statistical programming language [R](https://www.r-project.org/about.html). 

###Introduction to statsmodels

`statsmodels` is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration. 

`statsmodels` supports specifying models using R-style formulas and pandas DataFrames. 

Run the following cell to import the libraries we will explore in this lab assignment.

In [1]:
import pandas as pd #data manipulation
import numpy as np #array manipulation

import statsmodels.formula.api as smf #regression
from statsmodels.formula.api import ols, logit

import seaborn as sns #statistical plotting
import matplotlib.pyplot as plt #plotting

###Cease-Fire Agreements and the Durability of Peace

The Cease-Fires data set was developed for data analysis in [Peace Time: Cease-Fire Agreements and the Durability of Peace](https://press.princeton.edu/books/paperback/9780691115122/peace-time) to explore cease-fire agreements in the study of war and peace.

The data cover all cease-fires in interstate wars ending between 1946 and 1994. There are 48 basic cases in the data set, each representing a dyadic cease-fire between principal belligerents in a Correlates of War (COW) interstate war. 

There are two versions of the data, a time-constant covariates version, which is used primarily for analysis of agreements as the dependent variable, and a time-covarying variates version, which is used primarily for duration analysis with the duration of peace as the dependent variable.

The time-constant (tc) version `ceasefires.tc.dta` consists of the 48 original cease-fires. In this version, each case consists of a single observation, representing a snap-shot of the case.

In the time-varying (tv) version of the data `ceasefires.tv.dta`, there are 48 cases, but each case consists of multiple observations over time.

###1. The Data

####Question 1.1

The lab assignment will use the time-constant covariates version. Read in the Stata dta file `ceasefires.tc.dta` into a pandas dataframe with the name: `ceasefire`. You might find looking at [the documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_stata.html) helpful.

The `DataFrame.head()` method shows the first few lines of the dataframe. 

In [2]:
#Bring in the main analysis dataset
ceasefire = pd.read_stata("dataverse_files/ceasefires.tv.dta")
ceasefire.head()

,id,warnumb,war,ccode1,ccode2,cluster,date0,date1,newwar,cfyear,...,USmarker,marker,_st,_d,_origin,_t,_t0,cfhist,_Isettle_1,_Isettle_2
0,1.0,148.0,PALESTINE 1,666.0,645.0,1.0,1948-07-18,1948-10-22,1.0,1.0,...,X,X,1,1,-4184,96,0,1.0,0,0
1,2.0,148.0,PALESTINE 1,666.0,651.0,1.0,1948-07-18,1948-10-22,1.0,1.0,...,X,X,1,1,-4184,96,0,1.0,0,0
2,3.0,148.0,PALESTINE 1,666.0,652.0,1.0,1948-07-18,1948-10-22,1.0,1.0,...,X,X,1,1,-4184,96,0,1.0,0,0
3,4.0,148.0,PALESTINE 1,666.0,660.0,1.0,1948-07-18,1948-10-22,1.0,1.0,...,X,X,1,1,-4184,96,0,1.0,0,0
4,5.0,148.0,PALESTINE 1,666.0,663.0,1.0,1948-07-18,1948-10-22,1.0,1.0,...,X,X,1,1,-4184,96,0,1.0,0,0


####Question 1.2

Now that you've read in the dta file let's try some `pd.DataFrame` methods. To print out a list of all the column names in the dataframe use the `DataFrame.columns.values` method.

In [3]:
ceasefire.columns.values

array(['id', 'warnumb', 'war', 'ccode1', 'ccode2', 'cluster', 'date0',
       'date1', 'newwar', 'cfyear', 'followup', 'drop', 'tie', 'untie',
       'imposed', 'foreign', 'death1', 'death2', 'lndeaths', 's1_death',
       's2_death', 't_deaths', 'duration', 'disputes', 'dyadage',
       'cfhist_r', 'stakes', 'stake_ip', 'stake_t', 'stake_g', 'stake_e',
       'rev_type', 'rev_terr', 'multi', 'contig', 'gp_bel', 'perm5',
       'usbel', 'israel', 'cap_1', 'cap_2', 'prepond', 'prep_att',
       'maxcap', 'lagcap_1', 'lagcap_2', 'd_relcap', 'lagrelcp',
       'equilib', 'eudemand', 'euwar', 'dem1', 'dem2', 'onedem', 'twodem',
       'lagdem1', 'lagdem2', 'politych', 'newdem', 'i_lntime', 'formal',
       'formal_d', 'withdraw', 'with_dum', 'with_sqa', 'dmz', 'dmz_dum',
       'dmz_wide', 'ac', 'ac_dum', 'pk', 'pk_dum', 'pk_num', 'pk_who',
       'pk_pre', 'newpk', 'newpkdum', 'pkopC', 'pk_dumC', 'ext_inv',
       'internal', 'paragrph', 'detail', 'info', 'info_dum', 'disp_res',
       'c

As you can see the `ceasefire` dataframe has quite a few columns. Take a moment to read through them. The relevant columns for our binary regression are `'...'` and `'strength'`.

###2. Binary Regression

Binary logistic regression is used for predicting binary classes. For example, in cases where you want to predict yes/no, win/loss, negative/positive, True/False, admission/rejection and so on. 

Fitting a regression in python using `statsmodels` is similar to fitting it in R, as statsmodels package also supports formula like syntax.

Here we are going to use the [`logit`](https://www.statsmodels.org/stable/generated/statsmodels.formula.api.logit.html) model for model estimation.

####Question 2.1
`...` is the binary dependent variable in the `ceasefire` dataset with the categories N/A. 

Implement a logistic regression to predict the binary outcome- `...` in the dataset `ceasefire`. Analyze the model summary by using a `print(...)` statement.

In [4]:
logit_model = smf.logit(formula="durability~strength", data = ceasefire)
logit_estimates = logit_model.fit()
print(logit_estimates.summary())

PatsyError: Error evaluating factor: NameError: name 'durability' is not defined
    durability~strength
    ^^^^^^^^^^

####Question 2.X
Fill in the blank. It is recommended you discuss these questions with someone to make sure you both understand what binary regression is and when it is used.

Logistic Regression is a (machine learning classification) algorithm which is used when we want to (predict a categorical variable (Yes/No, Pass/Fail) based on a set of independent variables).

Binary regression requires the (dependent variable) to be binary.

The independent variables should be (independent) of each other. The model should have little or no multicollinearity.

The (independent) variables are linearly related to the log odds of the (dependent) variables.

####Question 2.X


####Resources
https://www.statsmodels.org/stable/index.html
https://towardsdatascience.com/implementing-binary-logistic-regression-in-r-7d802a9d98fe